In [1]:
import numpy as np

# Grid Geometry

In [2]:
grid_num = 4

In [3]:
grid_xedge = 1000./grid_num
grid_yedge = 200
grid_zedge = 10

In [4]:
grid_xarea = grid_yedge*grid_zedge

# Rock Properties

In [5]:
rock_poro = 0.2
rock_xperm = 100
rock_comp = 0

# Black Oil System

In [6]:
oil_visc = 1
oil_comp = 4e-6
oil_fvf = 1

In [7]:
wat_visc = 1
wat_comp = 1e-6
wat_fvf = 1

In [8]:
def krw(Sw):
  return 0.3*((Sw-0.2)/(1-0.2-0.4))**2

In [9]:
def kro(Sw):
  return 0.8*((1.0-Sw-0.4)/(1-0.2-0.4))**2

# Well Input

In [10]:
w1_block = (3,)
w1_radius = 0.25
w1_wrate = 1000
w1_skin = 0

In [11]:
w2_block = (0,)
w2_radius = 0.25
w2_press = 900
w2_skin = 0

# Time Settings

In [12]:
deltat = 1

# Reservoir Initialization

In [13]:
init_press = 1000 # not used in the calculations
init_swater = 0.2 # not used in the calculations

Conditions at certain time step:

In [14]:
Pn = np.array([1065.3,2187.1,1369.8,3075.4])

In [15]:
Swn = np.array([0.2002,0.2429,0.4346,0.5258])

# Transmissibility Calculations

Reservoir properties are constant, thus, we have only one xrock value and it is going to be the same after harmonic mean.

In [16]:
xrock = (rock_xperm*grid_xarea)/grid_xedge*6.33e-3; xrock

5.064

Mobility Calculations:

In [17]:
Mo = kro(Swn)/(oil_visc*oil_fvf); Mo

array([0.7992002 , 0.63760205, 0.1367858 , 0.0275282 ])

In [18]:
Mw = krw(Swn)/(wat_visc*wat_fvf); Mw

array([7.50000000e-08, 3.45076875e-03, 1.03194675e-01, 1.99023075e-01])

Upwinding:

In [19]:
Mo_upwinded = np.zeros((grid_num-1,))
Mw_upwinded = np.zeros((grid_num-1,))

for index in range(grid_num-1):
    if Pn[index]>Pn[index+1]:
        Mo_upwinded[index] = Mo[index]
        Mw_upwinded[index] = Mw[index]
    else:
        Mo_upwinded[index] = Mo[index+1]
        Mw_upwinded[index] = Mw[index+1]

In [20]:
Mo_upwinded

array([0.63760205, 0.63760205, 0.0275282 ])

In [21]:
Mw_upwinded

array([0.00345077, 0.00345077, 0.19902307])

Transmissibility values for both phases:

In [22]:
otrans = Mo_upwinded*xrock; otrans

array([3.22881678, 3.22881678, 0.1394028 ])

In [23]:
wtrans = Mw_upwinded*xrock; wtrans

array([0.01747469, 0.01747469, 1.00785285])

# Storage and Total Compressibility Calculations

In [24]:
accum = grid_xarea*grid_xedge*rock_poro/deltat; accum

100000.0

In [25]:
def get_ct(cr,co,cw,Sw):
  return cr+Sw*cw+(1-Sw)*co

In [26]:
ct = get_ct(rock_comp,oil_comp,wat_comp,Swn); ct

array([3.3994e-06, 3.2713e-06, 2.6962e-06, 2.4226e-06])

# Well B.C. Calculations

In [27]:
def equivalent_radius(perm1,perm2,edge1,edge2):
    """Returns equivalent radius of grids that contains well."""
    sqrt21 = np.power(perm2/perm1,1/2)*np.power(edge1,2)
    sqrt12 = np.power(perm1/perm2,1/2)*np.power(edge2,2)

    quar21 = np.power(perm2/perm1,1/4)
    quar12 = np.power(perm1/perm2,1/4)

    return 0.28*np.sqrt(sqrt21+sqrt12)/(quar21+quar12)

In [28]:
req = equivalent_radius(rock_xperm,rock_xperm,grid_xedge,grid_yedge)

In [29]:
w1_PI = (2*np.pi*grid_zedge*rock_xperm)/(np.log(req/w1_radius)+w1_skin)*6.33e-3
w2_PI = (2*np.pi*grid_zedge*rock_xperm)/(np.log(req/w2_radius)+w2_skin)*6.33e-3

In [30]:
w1_PI_o = w1_PI*Mo[w1_block]; w1_PI_o

0.21099808516503138

In [31]:
w1_PI_w = w1_PI*Mw[w1_block]; w1_PI_w

1.5254716156035082

In [32]:
w2_PI_o = w2_PI*Mo[w2_block]; w2_PI_o

6.1257078873122985

In [33]:
w2_PI_w = w2_PI*Mw[w2_block]; w2_PI_w

5.748598305509948e-07

# Matrix Construction

In [34]:
Tw = np.zeros((grid_num,grid_num))
To = np.zeros((grid_num,grid_num))

Jw = np.zeros((grid_num,grid_num))
Jo = np.zeros((grid_num,grid_num))

Qw = np.zeros((grid_num,1))
Qo = np.zeros((grid_num,1))

A = np.zeros((grid_num,grid_num))

Pwf = np.zeros((grid_num,1))

for i in range(grid_num):
    if i == 0:
        Tw[i,i] = wtrans[i]
        Tw[i,i+1] = -wtrans[i]
        To[i,i] = otrans[i]
        To[i,i+1] = -otrans[i]
        Jw[i,i] = w2_PI_w
        Jo[i,i] = w2_PI_o
        Qw[i,0] = Jw[i,i]*w2_press
        Qo[i,0] = Jo[i,i]*w2_press
        Pwf[i,0] = w2_press
    elif i == 3:
        Tw[i,i] = wtrans[i-1]
        Tw[i,i-1] = -wtrans[i-1]
        To[i,i] = otrans[i-1]
        To[i,i-1] = -otrans[i-1]
        Qw[i,0] = w1_wrate
    else:
        Tw[i,i] = wtrans[i]+wtrans[i-1]
        Tw[i,i+1] = -wtrans[i]
        Tw[i,i-1] = -wtrans[i-1]
        To[i,i] = otrans[i]+otrans[i-1]
        To[i,i+1] = -otrans[i]
        To[i,i-1] = -otrans[i-1]
    A[i,i] = grid_xarea*grid_xedge*rock_poro/deltat

In [35]:
Tw

array([[ 0.01747469, -0.01747469,  0.        ,  0.        ],
       [-0.01747469,  0.03494939, -0.01747469,  0.        ],
       [ 0.        , -0.01747469,  1.02532754, -1.00785285],
       [ 0.        ,  0.        , -1.00785285,  1.00785285]])

In [36]:
To

array([[ 3.22881678, -3.22881678,  0.        ,  0.        ],
       [-3.22881678,  6.45763356, -3.22881678,  0.        ],
       [ 0.        , -3.22881678,  3.36821959, -0.1394028 ],
       [ 0.        ,  0.        , -0.1394028 ,  0.1394028 ]])

In [37]:
Jw

array([[5.74859831e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [38]:
Jo

array([[6.12570789, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [39]:
Qw

array([[5.17373847e-04],
       [0.00000000e+00],
       [0.00000000e+00],
       [1.00000000e+03]])

In [40]:
Qo

array([[5513.13709858],
       [   0.        ],
       [   0.        ],
       [   0.        ]])

In [41]:
T = Tw+To; T

array([[ 3.24629147, -3.24629147,  0.        ,  0.        ],
       [-3.24629147,  6.49258295, -3.24629147,  0.        ],
       [ 0.        , -3.24629147,  4.39354713, -1.14725566],
       [ 0.        ,  0.        , -1.14725566,  1.14725566]])

In [42]:
Act = np.matmul(A,np.diag(ct)); Act

array([[0.33994, 0.     , 0.     , 0.     ],
       [0.     , 0.32713, 0.     , 0.     ],
       [0.     , 0.     , 0.26962, 0.     ],
       [0.     , 0.     , 0.     , 0.24226]])

In [43]:
J = Jw+Jo; J

array([[6.12570846, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [44]:
Q = Qw+Qo; Q

array([[5513.13761595],
       [   0.        ],
       [   0.        ],
       [1000.        ]])

# Solving Linear System of Equations

In [45]:
Pnew = np.linalg.solve(T+J+Act,np.matmul(Act,Pn.reshape((-1,1)))+Q); Pnew.flatten()

array([1096.28026655, 1469.90263878, 1771.25276601, 2718.30413795])

In [46]:
Swnew = Swn+np.linalg.solve(A,np.matmul(-Tw,Pnew)+Qw+np.matmul(Jw,Pwf-Pnew)).flatten()-Swn*(wat_comp+rock_comp)*(Pnew.flatten()-Pn); Swnew

array([0.20025909, 0.24306158, 0.44391775, 0.52644288])